Based on this paper:

Real-ESRGAN: Training Real-World Blind Super-Resolution with Pure Synthetic Data

https://arxiv.org/abs/2107.10833

# Instructions:

In [ ]:
# clone Real-ESRGAN Repo
!git clone https://github.com/xinntao/Real-ESRGAN.git

In [ ]:
%cd Real-ESRGAN

In [ ]:
# Install basicsr - https://github.com/xinntao/BasicSR
# We use BasicSR for both training and inference
!pip install basicsr
# facexlib and gfpgan are for face enhancement
!pip install facexlib
!pip install gfpgan
!pip install -r requirements.txt
!python setup.py develop

In [ ]:
import os
from PIL import Image
import numpy as np
from skimage.metrics import peak_signal_noise_ratio, structural_similarity
from torchvision.transforms.functional import rgb_to_grayscale

In [ ]:
test_dir = '../data/DIV2K_valid_LR_bicubic_X4_extracted/DIV2K_valid_LR_bicubic/X4/'
test_hr_dir = '../data/DIV2K_valid_HR_extracted/DIV2K_valid_HR'
results_dir = '../results'

In [ ]:
def upscale_image(image_path, model_path):
    """Upscales a single image using the Real-ESRGAN model."""
    upscaled_path = os.path.join(results_dir, os.path.basename(image_path))
    # -s 4 for x4 scale
    os.system(f'python inference_realesrgan.py -i {image_path} -o {results_dir} -n {model_path} -s 4')
    return upscaled_path

def evaluate_image(upscaled_path, hr_path):
    """Calculates PSNR and SSIM between upscaled and HR images."""
    upscaled_img = np.array(Image.open(upscaled_path).convert("RGB"))
    hr_img = np.array(Image.open(hr_path).convert("RGB"))

    psnr = peak_signal_noise_ratio(hr_img, upscaled_img, data_range=255)
    ssim = structural_similarity(hr_img, upscaled_img, multichannel=True, data_range=255)
    return psnr, ssim

In [ ]:
model_path = 'experiments/pretrained_models/RealESRGAN_x4plus.pth'

In [ ]:
psnr_values = []
ssim_values = []

for filename in os.listdir(test_dir):
    if filename.endswith(('.png')):
        lr_path = os.path.join(test_dir, filename)
        hr_filename = filename.replace('x4', '')
        hr_path = os.path.join(test_hr_dir, hr_filename)

        if os.path.exists(hr_path):
            print(f"Upscaling: {filename}")
            upscaled_path = upscale_image(lr_path, model_path)
            
            print(f"Evaluating: {filename}")
            psnr, ssim = evaluate_image(upscaled_path, hr_path)
            psnr_values.append(psnr)
            ssim_values.append(ssim)

            print(f"PSNR: {psnr:.2f}, SSIM: {ssim:.4f}")

In [ ]:
avg_psnr = np.mean(psnr_values)
avg_ssim = np.mean(ssim_values)

print(f"\nAverage PSNR: {avg_psnr:.2f}")
print(f"Average SSIM: {avg_ssim:.4f}")

In [ ]:
import cv2
import numpy as np
import os
import glob
import matplotlib.pyplot as plt
from pathlib import Path

### Preparing the Images

In [ ]:
test_dir = '../data/DIV2K_valid_LR_bicubic_X4_extracted/DIV2K_valid_LR_bicubic/X4/'
output_dir = '../ESRGAN_Test_Images/'

# create directories if they don't exist
os.makedirs(output_dir + "hr_images", exist_ok=True)
os.makedirs(output_dir + "lr_images", exist_ok=True)

def is_image_file(filename):
    return any(filename.endswith(extension) for extension in ['.png', '.jpg', '.jpeg', '.PNG', '.JPG', '.JPEG'])

for img in os.listdir(test_dir)[0:10]:
    if is_image_file(img):
        img_array = cv2.imread(test_dir + img)
        img_array = cv2.resize(img_array, (128,128))
        lr_img_array = cv2.resize(img_array,(32,32))
        cv2.imwrite(output_dir+ "hr_images/" + img, img_array)
        cv2.imwrite(output_dir+ "lr_images/"+ img, lr_img_array)

### Model Prediction

In [ ]:
model_path = Path('experiments/pretrained_models/RealESRGAN_x4plus.pth')
input_dir = Path('../ESRGAN_Test_Images/lr_images')
output_dir = Path('../results')

def create_directory(path):
    if not path.exists():
        os.makedirs(path)

create_directory(output_dir)

try:
    # Run inference command
    os.system(f"python inference_realesrgan.py -n {model_path} -i {input_dir} -o {output_dir} --outscale 8 --face_enhance")
    
    print("Inference completed successfully.")
except Exception as e:
    print(f"An error occurred: {str(e)}")

In [ ]:
!python inference_realesrgan.py -n RealESRGAN_x4plus -i '../ESRGAN_Test_Images/lr_images' -o '../results' --outscale 8 --face_enhance

### Image Visualization

In [ ]:
def display(img1, img2, img3):
    
  fig = plt.figure(figsize=(15, 5))

  ax1 = fig.add_subplot(1, 3, 1) 
  plt.title('Input image', fontsize=16)
  ax1.axis('off')
  ax2 = fig.add_subplot(1, 3, 2)
  plt.title('Real-ESRGAN output', fontsize=16)
  ax2.axis('off')
  ax3 = fig.add_subplot(1, 3, 3)
  plt.title('Original Image', fontsize=16)
  ax3.axis('off')
  ax1.imshow(img1)
  ax2.imshow(img2)
  ax3.imshow(img3)
  plt.show()

def imread(img_path):
  img = cv2.imread(img_path)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  return img

In [ ]:

input_folder = '../ESRGAN_Test_Images/lr_images'
original_folder = '../ESRGAN_Test_Images/hr_images'
result_folder = '../results'
input_list = sorted(glob.glob(os.path.join(input_folder, '*')))
original_list = sorted(glob.glob(os.path.join(original_folder, '*')))
output_list = sorted(glob.glob(os.path.join(result_folder, '*')))

demo_inp_list = input_list[0:5]
demo_original_list = original_list[0:5]
demo_out_list = output_list[0:5]


for input_path,original_path,output_path in zip(demo_inp_list,demo_original_list,demo_out_list):
  img_input = imread(input_path)
  img_output = imread(output_path)
  img_original = imread(original_path)
  display(img_input,img_output,img_original)